In [ ]:
from trading_engine.core import (
    read_data, create_model_state, orchestrate_model_backtests,
    orchestrate_model_simulations, orchestrate_portfolio_optimizations,
    orchestrate_portfolio_simulations, orchestrate_portfolio_aggregation
)

import datetime

In [ ]:
# 1) experiment config
universe = [
  "SPY-US", "SLV-US", "GLD-US", "TLT-US", "USO-US", "UNG-US", "IXJ-US",
  "KXI-US", "JXI-US", "IXG-US", "IXN-US", "RXI-US", "MXI-US", "EXI-US",
  "IXC-US", "IEI-US", "SHY-US", "BIL-US", "JPXN-US", "INDA-US", "MCHI-US",
  "EZU-US", "IBIT-US", "ETHA-US", "VIXY-US"
]
features = ["close_momentum_10"]                   # keys in FEATURES
models   = ["RXI_TLT_pml_10", "GLD_USO_nml_10"]    # keys in MODELS
aggregators = ["model_mvo"]                        # keys in AGGREGATORS
optimizers   = ["mean_variance_constrained"]       # keys in OPTIMIZERS
initial_value = 1_000_000
start_date = datetime.date(2021, 1, 1)
end_date = datetime.date(2025, 1, 1)

In [ ]:
# 2) build model state + prices (cached locally)
raw_data_bundle = read_data(include_supplemental=True)
model_state_bundle, prices = create_model_state(
    raw_data_bundle=raw_data_bundle,
    features=features,
    start_date=start_date,
    end_date=end_date,
    universe=universe
)

In [ ]:
# 3) run model backtests + simulations
model_insights = orchestrate_model_backtests(
    model_state_bundle=model_state_bundle,
    models=models,
    universe=universe
)

model_simulations = orchestrate_model_simulations(
    prices=prices,
    model_insights=model_insights,
    start_date=start_date,
    end_date=end_date
)

In [ ]:
# 4) aggregate + optimize portfolio and simulate
aggregated_insights = orchestrate_portfolio_aggregation(
    model_insights=model_insights,
    backtest_results=model_simulations,
    universe=universe,
    aggregators=aggregators,
    start_date=start_date,
    end_date=end_date,
)

optimizer_insights = orchestrate_portfolio_optimizations(
    prices=prices,
    aggregated_insights=aggregated_insights,
    universe=universe,
    optimizers=optimizers,
)

optimizer_simulations = orchestrate_portfolio_simulations(
    prices=prices,
    portfolio_insights=optimizer_insights,
    start_date=start_date,
    end_date=end_date,
    initial_value=initial_value,
)

In [ ]:
# 5) visualize one result (example: mean_variance_constrained)
optimizer_simulations["mean_variance_constrained"]["backtest_metrics"]